# SACC file example for binned cluster analysis

This demo notebook examplifies storing information for a binned cluster analysis into a sacc format. Steps are the following:
- generate a mock cluster catalog in richness-redshift bins. This is done using NumCosmo functionalities and requires Numcosmo v>=0.17
- store all this information using the SACC framework. We present to cases:
    - store **number counts and mean mass** in a single SACC file;
    - store **number counts and radial shear profile** in a single SACC file

For the latter, in each richness-redshift bin, we generate a mock radial shear profile using CLMM. This is meant to correspond to the stacked shear profile in that bin.

This should be straightforward to adapt to store counts, mean mass and radial profile in individual sacc file instead.

In [ ]:
import os
import math
import itertools
import numpy as np
from astropy.table import Table
from astropy.io import fits
from scipy import stats
import matplotlib.pyplot as plt

# This require Numcosmo v>=0.17
from numcosmo_py import Nc
from numcosmo_py import Ncm

from firecrown.sacc_support import sacc

os.environ[
    "CLMM_MODELING_BACKEND"
] = "nc"  # Need to use NumCosmo as CLMM's backend as well.
import clmm
from clmm import Cosmology

## Prepare NumCosmo cosmology

In [ ]:
H0 = 71.0
Ob0 = 0.0448
Odm0 = 0.22
n_s = 0.963
sigma8 = 0.8

Ncm.cfg_init()
# cosmo = Nc.HICosmoDEXcdm() # not (yet) supported by CLMM --> using Nc.HICosmoDECpl() instead
cosmo = Nc.HICosmoDECpl()
reion = Nc.HIReionCamb.new()
prim = Nc.HIPrimPowerLaw.new()

cosmo.add_submodel(reion)
cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new(tf)

# psml = Nc.PowspecMLCBE.new ()
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

cosmo.props.H0 = H0
cosmo.props.Omegab = Ob0
cosmo.props.Omegac = Odm0

cosmo.omega_x2omega_k()
cosmo.param_set_by_name("Omegak", 0.0)

prim.props.n_SA = n_s

old_amplitude = math.exp(prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log((sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude)

## Generate a mock cluster catalog using NumCosmo

NumCosmo has a built-in functionality to generate mock cluster catalogs, from a given halo mass function (HMF) and with a log-normal richness-mass relation. 
- For the HMF, the code below uses the Tinker (2008) definition. 
- For the richness-mass relation, six parameters can be specified as the NumCosmo functionality allows for the mean relation and scatter of the log-normal distribution to have a linear dependences with both log(mass) and log(1+z) (that's 2 parameters and a normalisation, for both the mean relation and the scatter). 

Below we use the Murata et al. (2019) parameters (from their Table 1), but neglecting the quadratic dependence in redshift they use (this is inconsistent, but sufficient to generate this mock dataset)

In [ ]:
# Define the sky area and the richness and redshift ranges
area = 5000.0  # deg2
lnRl = 0.0
lnRu = 6.0
zl = 0.2
zu = 0.65

cluster_z = Nc.ClusterRedshift.new_from_name(
    f"NcClusterRedshiftNodist{{'z-min': <{zl:22.15e}>, 'z-max':<{zu:22.15e}>}}"
)

cluster_m = Nc.ClusterMass.new_from_name(
    f"NcClusterMassAscaso{{'M0':<{3.0e14 / 0.71:22.15e}>,'z0':<0.6>, "
    f"'lnRichness-min':<{lnRl:22.15e}>, 'lnRichness-max':<{lnRu:22.15e}>}}"
)

# mean richness-mass relation parameters
cluster_m.param_set_by_name("mup0", 3.15)  # normalisation
cluster_m.param_set_by_name(
    "mup1", 0.86 / np.log(10)
)  # mass dependence, adapted to match the log10 definition used in NumCosmo
cluster_m.param_set_by_name(
    "mup2", -0.21 / np.log(10)
)  # redshift dependence, adapted to match the log10 definition used in NumCosmo
# richness-mass scatter parameter
cluster_m.param_set_by_name("sigmap0", 0.33)  # normalisation
cluster_m.param_set_by_name(
    "sigmap1", -0.08 / np.log(10)
)  # mass dependence, adapted to match the log10 definition used in NumCosmo
cluster_m.param_set_by_name(
    "sigmap2", 0.03 / np.log(10)
)  # redshift dependence, adapted to match the log10 definition used in NumCosmo


# Numcosmo Mass Function
# First, define the multiplicity function.
mulf = Nc.MultiplicityFuncTinker.new()  # Tinker (2008)
mulf.set_linear_interp(True)  # This reproduces the linear interpolation done in CCL
mulf.set_mdef(Nc.MultiplicityFuncMassDef.MEAN)
mulf.set_Delta(200)

# Second, construct a filtered power spectrum
hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area_sd(area)

# Cluster Abundance Obj
ca = Nc.ClusterAbundance.new(hmf, None)

# Number Counts object
ncount = Nc.DataClusterNCount.new(ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ca.prepare(cosmo, cluster_z, cluster_m)
mset = Ncm.MSet.new_array([cosmo, cluster_z, cluster_m])

rng = Ncm.RNG.seeded_new(None, 32)
ncount.init_from_sampling(mset, area * ((np.pi / 180) ** 2), rng)

ncount.catalog_save("ncount_rich.fits", True)
ncdata_fits = fits.open("ncount_rich.fits")
ncdata_data = ncdata_fits[1].data
ncdata_Table = Table(ncdata_data)

## SACC for number counts and mean mass in the $N_{\rm richness} \times N_z$ richness-redshift plane
- count halos and compute mean mass in each bin, directly from the data generated at the previous step
- associate a "mock" shear profile, generated using CLMM from the mean mass in the bin, mock diagonal inter-radial bin covariance. Very rough/quick solution to get some "shear profile data".


### Counts and mean mass


In [ ]:
data_table = ncdata_Table[ncdata_Table["LNM_OBS"] > 2]
cluster_z = data_table["Z_OBS"]
cluster_lnm = data_table["LNM_OBS"]
cluster_richness = cluster_lnm / np.log(10.0)
cluster_logM = np.log10(np.exp(data_table["LNM_TRUE"]))

N_richness = 4  # number of richness bins
N_z = 3  # number of redshift bins

cluster_counts, z_edges, richness_edges, _ = stats.binned_statistic_2d(
    cluster_z, cluster_richness, cluster_logM, "count", bins=[N_z, N_richness]
)

mean_logM = stats.binned_statistic_2d(
    cluster_z,
    cluster_richness,
    cluster_logM,
    "mean",
    bins=[z_edges, richness_edges],
).statistic

std_logM = stats.binned_statistic_2d(
    cluster_z, cluster_richness, cluster_logM, "std", bins=[z_edges, richness_edges]
).statistic

var_mean_logM = std_logM**2 / cluster_counts

covariance = np.diag(
    np.concatenate((cluster_counts.flatten(), var_mean_logM.flatten()))
)

### Saving to SACC

In [ ]:
s_count = sacc.Sacc()
bin_z_labels = []
bin_richness_labels = []

survey_name = "NC_mock_redshift_richness"
s_count.add_tracer("cluster_survey", survey_name, area)

for i, z_bin in enumerate(zip(z_edges[:-1], z_edges[1:])):
    lower, upper = z_bin
    bin_z_label = f"bin_z_{i}"
    s_count.add_tracer("bin_z", bin_z_label, lower, upper)
    bin_z_labels.append(bin_z_label)

for i, richness_bin in enumerate(zip(richness_edges[:-1], richness_edges[1:])):
    lower, upper = richness_bin
    bin_richness_label = f"bin_rich_{i}"
    s_count.add_tracer("bin_richness", bin_richness_label, lower, upper)
    bin_richness_labels.append(bin_richness_label)

cluster_count = sacc.standard_types.cluster_counts
cluster_mass = sacc.standard_types.cluster_mean_log_mass

counts_and_edges = zip(
    cluster_counts.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)

# mean_logM_and_edges = zip(
#     mean_logM.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
# )

mean_logM_and_edges = zip(
    mean_logM.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)


for counts, (bin_z_label, bin_richness_label) in counts_and_edges:
    s_count.add_data_point(
        cluster_count, (survey_name, bin_z_label, bin_richness_label), int(counts)
    )

for bin_mean_logM, (bin_z_label, bin_richness_label) in mean_logM_and_edges:
    s_count.add_data_point(
        cluster_mass,
        (survey_name, bin_z_label, bin_richness_label),
        bin_mean_logM,
    )

# Then the add the covariance and save the file
s_count.add_covariance(covariance)
s_count.to_canonical_order()
s_count.save_fits("cluster_redshift_richness_sacc_data.fits", overwrite=True)

### Loading SACC file and check contents

In [ ]:
s2 = sacc.Sacc.load_fits("./cluster_redshift_richness_sacc_data.fits")

In [ ]:
s2.get_data_types()

In [ ]:
s2.data

#### Look at covariance. 
Plotting log10 of the covariance for display purposes. The first upper left block correspond to the count covariance. 
The lower right block to the mean mass covariance. No off-diagonal terms were considered in this simple example. This is just a place holder to show how the SACC file is filled in.

In [ ]:
plt.imshow(np.log10(s2.covariance.covmat))
plt.colorbar()

## SACC for number counts and shear profile in the $N_{\rm richness} \times N_z$ richness-redshift plane


Rather than saving the mean mass into SACC, one may want to save the 'stacked' shear profile in a richness-redshift bin. This possible as exemplified below. 

### Prepare new sacc, filling it with counts (same as above)

In [ ]:
s_count2 = sacc.Sacc()
bin_z_labels = []
bin_richness_labels = []

survey_name = "NC_mock_redshift_richness"
s_count2.add_tracer("cluster_survey", survey_name, area)

for i, z_bin in enumerate(zip(z_edges[:-1], z_edges[1:])):
    lower, upper = z_bin
    bin_z_label = f"bin_z_{i}"
    s_count2.add_tracer("bin_z", bin_z_label, lower, upper)
    bin_z_labels.append(bin_z_label)

for i, richness_bin in enumerate(zip(richness_edges[:-1], richness_edges[1:])):
    lower, upper = richness_bin
    bin_richness_label = f"bin_rich_{i}"
    s_count2.add_tracer("bin_richness", bin_richness_label, lower, upper)
    bin_richness_labels.append(bin_richness_label)

cluster_count = sacc.standard_types.cluster_counts
counts_and_edges = zip(
    cluster_counts.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)
for counts, (bin_z_label, bin_richness_label) in counts_and_edges:
    s_count2.add_data_point(
        cluster_count, (survey_name, bin_z_label, bin_richness_label), int(counts)
    )

### Use CLMM to generate a mock shear profile

In [ ]:
# Setting things up for CLMM
cosmo_clmm = Cosmology()
cosmo_clmm._init_from_cosmo(cosmo)
moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")
moo.set_cosmo(cosmo_clmm)
# assuming the same concentration for all masses. Not realistic, but avoid having to call a mass-concentration relation.
moo.set_concentration(4)

# we'll need the mean redshift of the clusters in the redshift bin
mean_z = stats.binned_statistic_2d(
    cluster_z,
    cluster_richness,
    cluster_z,
    "mean",
    bins=[z_edges, richness_edges],
).statistic

radius_edges = clmm.make_bins(
    0.3, 6.0, nbins=6, method="evenlog10width"
)  # 6 radial bins log-spaced between 0.3 and 6 Mpc

### Add the shear profile values to the SACC file

To do so, on top of the redshift bin tracer and the richness bin tracer already used for the counts and mean mass, we add a radial bin tracer. A shear profile data point will then depend on those three tracers.

In [ ]:
bin_radius_labels = []
radius_centers = []
for i, radius_bin in enumerate(zip(radius_edges[:-1], radius_edges[1:])):
    radius_lower, radius_upper = radius_bin
    radius_center = np.mean(radius_edges[i : i + 1])
    radius_centers.append(radius_center)
    bin_radius_label = f"bin_radius_{i}"
    s_count2.add_tracer(
        "bin_radius", bin_radius_label, radius_lower, radius_upper, radius_center
    )
    bin_radius_labels.append(bin_radius_label)

cluster_shear = sacc.standard_types.cluster_shear

Then we use CLMM to generate a mock shear profile in each richness and redshift bin, using the mean cluster mass and mean redhift in the bin as input.

In [ ]:
redshifts_masses_and_edges = zip(
    mean_z.flatten(),
    mean_logM.flatten(),
    itertools.product(bin_z_labels, bin_richness_labels),
)
for redshift, log_mass, (bin_z_label, bin_richness_label) in redshifts_masses_and_edges:
    mass = 10**log_mass
    moo.set_mass(mass)
    profile = moo.eval_excess_surface_density(radius_centers, redshift)
    for i, bin_radius_label in enumerate(bin_radius_labels):
        s_count2.add_data_point(
            cluster_shear,
            (survey_name, bin_z_label, bin_richness_label, bin_radius_label),
            profile[i],
        )

This is what the data now looks like:
- 4 richness x 3 redshift bins "number counts" datapoints
- 4 richness x 3 redshift x 6 radius bins "shear profile" datapoints

In [ ]:
s_count2.data

### Santity check - plotting the shear profile for two richness bins, to check that the high richness bin as higher shear (i.e. ordering of data is OK)

In [ ]:
DS0 = [
    s_count2.get_data_points(
        data_type="cluster_shear",
        tracers=(
            "NC_mock_redshift_richness",
            "bin_z_1",
            "bin_rich_0",
            f"bin_radius_{i}",
        ),
    )[0].value
    for i in np.arange(len(radius_edges) - 1)
]
DS2 = [
    s_count2.get_data_points(
        data_type="cluster_shear",
        tracers=(
            "NC_mock_redshift_richness",
            "bin_z_1",
            "bin_rich_2",
            f"bin_radius_{i}",
        ),
    )[0].value
    for i in np.arange(len(radius_edges) - 1)
]
r_arr = [
    s_count2.get_tracer(f"bin_radius_{i}").center
    for i in np.arange(len(radius_edges) - 1)
]

In [ ]:
plt.loglog(r_arr, DS0, label="bin_rich_0", marker="x")
plt.loglog(r_arr, DS2, label="bin_rich_2", marker="x")
plt.xlabel("R [Mpc]")
plt.ylabel("$\Delta\Sigma$ [M$_\odot \;$Mpc$^{-2}$]")
plt.legend()